Bina model klasifikasi: Masakan Asia dan India yang Lazat


## Pengelasan Masakan 2

Dalam pelajaran pengelasan kedua ini, kita akan meneroka `lebih banyak cara` untuk mengelaskan data kategori. Kita juga akan mempelajari implikasi memilih satu pengelas berbanding yang lain.

### [**Kuiz Pra-Kuliah**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Prasyarat**

Kami mengandaikan bahawa anda telah menyelesaikan pelajaran sebelumnya kerana kita akan membawa beberapa konsep yang telah dipelajari sebelum ini.

Untuk pelajaran ini, kita memerlukan pakej berikut:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) adalah [koleksi pakej R](https://www.tidyverse.org/packages) yang direka untuk menjadikan sains data lebih pantas, mudah dan menyeronokkan!

-   `tidymodels`: Kerangka [tidymodels](https://www.tidymodels.org/) adalah [koleksi pakej](https://www.tidymodels.org/packages/) untuk pemodelan dan pembelajaran mesin.

-   `themis`: Pakej [themis](https://themis.tidymodels.org/) menyediakan Langkah Resipi Tambahan untuk Menangani Data Tidak Seimbang.

Anda boleh memasangnya seperti berikut:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Sebagai alternatif, skrip di bawah akan memeriksa sama ada anda mempunyai pakej yang diperlukan untuk menyelesaikan modul ini dan memasangnya untuk anda jika ia tiada.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Peta pengelasan**

Dalam [pelajaran sebelumnya](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), kita cuba menjawab soalan: bagaimana kita memilih antara pelbagai model? Sebahagian besarnya bergantung pada ciri-ciri data dan jenis masalah yang ingin kita selesaikan (contohnya pengelasan atau regresi?)

Sebelumnya, kita telah mempelajari tentang pelbagai pilihan yang anda ada ketika mengelaskan data menggunakan helaian rujukan Microsoft. Rangka kerja Pembelajaran Mesin Python, Scikit-learn, menawarkan helaian rujukan yang serupa tetapi lebih terperinci yang boleh membantu anda menyempitkan pilihan estimator anda (istilah lain untuk pengelas):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Petua: [lawati peta ini secara dalam talian](https://scikit-learn.org/stable/tutorial/machine_learning_map/) dan klik sepanjang laluan untuk membaca dokumentasi.  
>  
> [Laman rujukan Tidymodels](https://www.tidymodels.org/find/parsnip/#models) juga menyediakan dokumentasi yang sangat baik tentang pelbagai jenis model.

### **Rancangan** 🗺️

Peta ini sangat berguna apabila anda mempunyai pemahaman yang jelas tentang data anda, kerana anda boleh 'berjalan' di sepanjang laluannya untuk membuat keputusan:

-   Kami mempunyai \>50 sampel

-   Kami ingin meramal kategori

-   Kami mempunyai data berlabel

-   Kami mempunyai kurang daripada 100K sampel

-   ✨ Kami boleh memilih Linear SVC

-   Jika itu tidak berjaya, kerana kami mempunyai data berangka

    -   Kami boleh mencuba ✨ KNeighbors Classifier

        -   Jika itu juga tidak berjaya, cuba ✨ SVC dan ✨ Ensemble Classifiers

Ini adalah laluan yang sangat berguna untuk diikuti. Sekarang, mari kita teruskan dengan menggunakan rangka kerja pemodelan [tidymodels](https://www.tidymodels.org/): satu koleksi pakej R yang konsisten dan fleksibel yang dibangunkan untuk menggalakkan amalan statistik yang baik 😊.

## 2. Bahagikan data dan tangani set data yang tidak seimbang.

Daripada pelajaran sebelumnya, kita belajar bahawa terdapat satu set bahan biasa merentasi masakan kita. Selain itu, terdapat pengagihan yang agak tidak seimbang dalam bilangan masakan.

Kita akan menangani perkara ini dengan

-   Membuang bahan yang paling biasa yang mencetuskan kekeliruan antara masakan yang berbeza, menggunakan `dplyr::select()`.

-   Menggunakan `recipe` yang memproses data untuk bersedia untuk pemodelan dengan menggunakan algoritma `over-sampling`.

Kita sudah melihat perkara di atas dalam pelajaran sebelumnya, jadi ini sepatutnya mudah 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Menangani Data Tidak Seimbang

Data tidak seimbang sering memberi kesan negatif terhadap prestasi model. Kebanyakan model berfungsi dengan baik apabila bilangan pemerhatian adalah sama dan, oleh itu, cenderung menghadapi kesukaran dengan data yang tidak seimbang.

Terdapat dua cara utama untuk menangani set data tidak seimbang:

-   menambah pemerhatian kepada kelas minoriti: `Over-sampling` contohnya menggunakan algoritma SMOTE yang secara sintetik menghasilkan contoh baharu bagi kelas minoriti menggunakan jiran terdekat bagi kes-kes ini.

-   mengurangkan pemerhatian daripada kelas majoriti: `Under-sampling`

Dalam pelajaran sebelumnya, kami telah menunjukkan cara menangani set data tidak seimbang menggunakan `recipe`. Recipe boleh dianggap sebagai pelan tindakan yang menerangkan langkah-langkah yang perlu diterapkan pada set data untuk menjadikannya sedia untuk analisis data. Dalam kes kita, kita mahu mempunyai pengagihan yang sama dalam bilangan masakan kita untuk `training set` kita. Mari kita teruskan.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Sekarang kita bersedia untuk melatih model 👩‍💻👨‍💻!

## 3. Melangkaui model regresi multinomial

Dalam pelajaran sebelumnya, kita telah melihat model regresi multinomial. Mari kita terokai beberapa model yang lebih fleksibel untuk klasifikasi.

### Support Vector Machines

Dalam konteks klasifikasi, `Support Vector Machines` adalah teknik pembelajaran mesin yang cuba mencari *hyperplane* yang "paling baik" memisahkan kelas-kelas. Mari kita lihat contoh mudah:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ tidak memisahkan kelas. H2~ memisahkan, tetapi hanya dengan margin kecil. H3~ memisahkan dengan margin maksimum.

#### Pengelas Linear Support Vector

Support-Vector clustering (SVC) adalah sebahagian daripada keluarga teknik ML Support-Vector machines. Dalam SVC, hyperplane dipilih untuk memisahkan `kebanyakan` pemerhatian latihan dengan betul, tetapi `mungkin salah klasifikasi` beberapa pemerhatian. Dengan membenarkan beberapa titik berada di sisi yang salah, SVM menjadi lebih tahan terhadap outlier dan dengan itu memberikan generalisasi yang lebih baik kepada data baharu. Parameter yang mengawal pelanggaran ini dirujuk sebagai `cost` yang mempunyai nilai lalai 1 (lihat `help("svm_poly")`).

Mari kita cipta SVC linear dengan menetapkan `degree = 1` dalam model SVM polinomial.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Sekarang kita telah menangkap langkah-langkah prapemprosesan dan spesifikasi model ke dalam *workflow*, kita boleh terus melatih SVC linear dan menilai hasilnya pada masa yang sama. Untuk metrik prestasi, mari kita cipta satu set metrik yang akan menilai: `accuracy`, `sensitivity`, `Positive Predicted Value` dan `F Measure`.

> `augment()` akan menambah lajur untuk ramalan kepada data yang diberikan.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Mesin Vektor Sokongan

Mesin vektor sokongan (SVM) adalah lanjutan daripada pengklasifikasi vektor sokongan untuk menyesuaikan sempadan tidak linear antara kelas. Secara asasnya, SVM menggunakan *helah kernel* untuk memperluaskan ruang ciri bagi menyesuaikan hubungan tidak linear antara kelas. Salah satu fungsi kernel yang popular dan sangat fleksibel yang digunakan oleh SVM ialah *Fungsi asas radial.* Mari kita lihat bagaimana ia berprestasi pada data kita.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Lebih baik 🤩!

> ✅ Sila lihat:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning dengan R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> untuk bacaan lanjut.

### Pengelas Neighbor Terdekat

*K*-nearest neighbor (KNN) adalah algoritma di mana setiap pemerhatian diramal berdasarkan *keserupaan* dengan pemerhatian lain.

Mari kita sesuaikan satu dengan data kita.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Model ini nampaknya tidak berfungsi dengan baik. Mungkin dengan mengubah argumen model (lihat `help("nearest_neighbor")`) akan meningkatkan prestasi model. Pastikan untuk mencubanya.

> ✅ Sila lihat:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> untuk mengetahui lebih lanjut tentang pengklasifikasi *K*-Nearest Neighbors.

### Pengklasifikasi Ensemble

Algoritma ensemble berfungsi dengan menggabungkan beberapa penganggar asas untuk menghasilkan model yang optimum sama ada melalui:

`bagging`: menggunakan *fungsi purata* pada koleksi model asas

`boosting`: membina urutan model yang saling melengkapi untuk meningkatkan prestasi ramalan.

Mari kita mulakan dengan mencuba model Random Forest, yang membina koleksi besar pokok keputusan kemudian menggunakan fungsi purata untuk menghasilkan model keseluruhan yang lebih baik.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Kerja yang bagus 👏!

Mari kita juga cuba dengan model Boosted Tree.

Boosted Tree mendefinisikan kaedah ensemble yang mencipta satu siri pokok keputusan secara berurutan di mana setiap pokok bergantung pada hasil pokok sebelumnya dalam usaha untuk mengurangkan ralat secara beransur-ansur. Ia memberi tumpuan kepada berat item yang diklasifikasikan secara salah dan menyesuaikan padanan untuk pengklasifikasi seterusnya bagi membetulkan.

Terdapat pelbagai cara untuk memadankan model ini (lihat `help("boost_tree")`). Dalam contoh ini, kita akan memadankan Boosted trees melalui enjin `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Sila lihat:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Membincangkan model AdaBoost yang merupakan alternatif yang baik kepada xgboost.
>
> untuk mempelajari lebih lanjut tentang pengelas Ensemble.

## 4. Tambahan - membandingkan pelbagai model

Kita telah melatih sejumlah besar model dalam makmal ini 🙌. Ia boleh menjadi membosankan atau membebankan untuk mencipta banyak aliran kerja daripada pelbagai set pemprosesan awal dan/atau spesifikasi model, kemudian mengira metrik prestasi satu persatu.

Mari kita lihat jika kita boleh menyelesaikan ini dengan mencipta satu fungsi yang melatih senarai aliran kerja pada set latihan, kemudian mengembalikan metrik prestasi berdasarkan set ujian. Kita akan menggunakan `map()` dan `map_dfr()` daripada pakej [purrr](https://purrr.tidyverse.org/) untuk menerapkan fungsi kepada setiap elemen dalam senarai.

> Fungsi [`map()`](https://purrr.tidyverse.org/reference/map.html) membolehkan anda menggantikan banyak gelung for dengan kod yang lebih ringkas dan mudah dibaca. Tempat terbaik untuk mempelajari fungsi [`map()`](https://purrr.tidyverse.org/reference/map.html) adalah dalam [bab iterasi](http://r4ds.had.co.nz/iteration.html) dalam buku R for Data Science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) membolehkan pengguna mencipta dan dengan mudah melatih sejumlah besar model, tetapi ia kebanyakannya direka untuk digunakan bersama teknik pensampelan semula seperti `cross-validation`, satu pendekatan yang kita belum bincangkan.

## **🚀Cabaran**

Setiap teknik ini mempunyai sejumlah besar parameter yang boleh anda ubah suai, contohnya `cost` dalam SVM, `neighbors` dalam KNN, `mtry` (Predictor yang Dipilih Secara Rawak) dalam Random Forest.

Selidik parameter lalai untuk setiap satu dan fikirkan apa maksud mengubah suai parameter ini terhadap kualiti model.

Untuk mengetahui lebih lanjut tentang model tertentu dan parameternya, gunakan: `help("model")` contohnya `help("rand_forest")`

> Dalam amalan, kita biasanya *menganggar* *nilai terbaik* untuk parameter ini dengan melatih banyak model pada `set data simulasi` dan mengukur sejauh mana prestasi semua model ini. Proses ini dipanggil **tuning**.

### [**Kuiz selepas kuliah**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Ulasan & Kajian Kendiri**

Terdapat banyak istilah teknikal dalam pelajaran ini, jadi luangkan masa untuk menyemak [senarai ini](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) istilah berguna!

#### TERIMA KASIH KEPADA:

[`Allison Horst`](https://twitter.com/allison_horst/) kerana mencipta ilustrasi yang menakjubkan yang menjadikan R lebih mesra dan menarik. Cari lebih banyak ilustrasi di [galerinya](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) dan [Jen Looper](https://www.twitter.com/jenlooper) kerana mencipta versi asal modul ini dalam Python ♥️

Selamat Belajar,

[Eric](https://twitter.com/ericntay), Duta Pelajar Microsoft Learn Emas.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Karya seni oleh @allison_horst</figcaption>



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat penting, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
